<a href="https://colab.research.google.com/github/BotenAqua/WarframeInventoryHelper/blob/master/WarframeInventoryHelper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [77]:
def htmlTableExtractor(url):
  #help/comment here
  #error handling here
  import requests
  #404 error handling here
  page = requests.get(url).text
  page = restrExtractor(page, ["<table", "</table>"])[0]

  if page.find("<th") > 0: #if table header exists
    tmp = restrExtractor(page, ["<tr", "</tr>"])
    page = page[tmp[1]:]
    header = []
    tmp = tmp[0]
    for colNo in range(tmp.find("<th")+1):
      name = restrExtractor(tmp, ['">', '</th>'])
      #print(name)
      #header.append(name[0]) w razie problemów z innymi stronami to dziala
      header.append(name[0][2:-6]) # to działa dla Warframe Wiki
      tmp = tmp[name[1]:]
    del tmp, name
    #print(header) #header is working fine

  tableContent = []
  for rowNo in range(page.count("<tr")):
    #print(rowNo)
    row = []
    row.append(restrExtractor(page, ["<tr", "</tr>"]))
    page = page[row[0][1]:]
    row[0] = row[0][0]
    for colNo in range(row[0].count("<td")):
      data = restrExtractor(row[0], ["<td", "</td>"])
      row.append(data[0][4:-5]) # tu kroje <td> i </td> jakby cos sie zepsulo
      row[0] = row[0][data[1]:]
      #print(row)
    row.pop(0)
    tableContent.append(row)
  return header, tableContent

def restrExtractor(page, restr):
  #help/comment here
  do = page.find(restr[1])+len(restr[1])
  return page[page.find(restr[0]):do], do

def warframeWikiScrapper():
  head, body = htmlTableExtractor("https://warframe.fandom.com/wiki/Void_Relic/ByRewards/SimpleTable")
  head.append("url")
  for l in range(len(body)):
    #dodanie URL
    adr = restrExtractor(body[l][0], ["/", " t"])
    body[l].append(adr[0][:-3])
    #czyszczenie nazwy
    body[l][0] = restrExtractor(body[l][0], ['">', '</'])[0][2:-2]
    #czyszczenie reliktu
    body[l][3] = restrExtractor(body[l][3], ["<a", "</a>"])[0]
    body[l][3] = restrExtractor(body[l][3], ['">', "</"])[0][2:-2] #jest sens zamykac w funkcje? :thinking:
    #czyszczenie vaulted?
    body[l][5] = body[l][5][:-1]
  return head, body[-1]

warframeWikiScrapper()

(['Item', 'Part', 'Tier', 'Name', 'Rarity', 'Vaulted?', 'url'],
 ['Zhuge Prime', 'String', 'Axi', 'M1', 'Common', 'No', '/wiki/Zhuge_Prime'])